In [1]:
import pandas as pd
import torch

from netam import framework, models
from netam.common import nt_mask_tensor_of, BASES

In [2]:
sim_row_count = 10000
shmoof_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
crepe_path = "../train/trained_models/cnn_joi_lrg-shmoof_small"
pcp_df = pd.read_csv(shmoof_path, index_col=0)
pcp_df = pcp_df[pcp_df["parent"] != pcp_df["child"]]
crepe = framework.load_crepe(crepe_path)

len(pcp_df)

49016

In [3]:
def mutation_count_of_row(row):    
    return sum(
        1 for p, c in zip(row["parent"], row["child"]) if p != c and p != "N" and c != "N"
    )    

pcp_df['mutation_count'] = pcp_df.apply(mutation_count_of_row, axis=1)
sim_df = pcp_df.sample(sim_row_count)[["parent", "branch_length", "mutation_count"]].reset_index(drop=True)
sim_df["parent_mask"] = sim_df["parent"].apply(lambda x: nt_mask_tensor_of(x, 500))

ratess, csp_logitss = crepe(sim_df["parent"])

sim_df.loc[0, "parent_mask"]

children = []

# iterate over rows of sim_df
for i, row in sim_df.iterrows():
    parent = row["parent"]
    branch_length = row["branch_length"]
    mutation_count = row["mutation_count"]
    parent_mask = row["parent_mask"]
    parent_mask = row["parent_mask"]
    rates = ratess[i]
    csp_logits = csp_logitss[i]
    rates *= parent_mask
    mut_idxs = torch.multinomial(rates, mutation_count, replacement=False)
    child = list(parent)
    for i in mut_idxs:
        child[i] = BASES[torch.multinomial(torch.exp(csp_logits[i]), 1)]
    children.append("".join(child))

sim_df["child"] = children
sim_df["simulated_mut_count"] = sim_df.apply(mutation_count_of_row, axis=1)
assert (sim_df["simulated_mut_count"] == sim_df["mutation_count"]).all()
# drop mutation_count and parent_mask and simulated_mut_count from pcp_df
sim_df = sim_df.drop(columns=["mutation_count", "parent_mask", "simulated_mut_count"])
sim_df.to_csv("mimic_shmoof_CNNJoiLrgShmoofSmall.10K.csv")
    